# A Prototype For Fitting Monotonic Cubic Splines to a Tdigest Sketch

### Register Custom Maven Resolvers

In [ ]:
import Resolvers._
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "manyangled",
  "https://dl.bintray.com/manyangled/maven/",
  MavenPattern,
  true
)
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
    "cibotech",
    "https://dl.bintray.com/cibotech/public/",
    MavenPattern,
    true
)

### Package imports

In [ ]:
// Apache Commons Math
import $ivy.`org.apache.commons:commons-math3:3.6.1`

// T-digest sketching
import $ivy.`org.isarnproject::isarn-sketches:0.1.2`

// Convex Optimization
import $ivy.`com.manyangled:gibbous:0.1.1`

// Monotonic splining
import $ivy.`com.manyangled:snowball:0.1.1`

// EvilPlot plotting routines
import $ivy.`com.cibo::evilplot:0.4.1`
import $ivy.`com.cibo::evilplot-repl:0.4.1`

### Imports for EvilPlot plotting support

In [ ]:
import com.cibo.evilplot._
import com.cibo.evilplot.plot._
import com.cibo.evilplot.plot.aesthetics.DefaultTheme._
import com.cibo.evilplot.numeric.Point
import com.cibo.evilplot.colors.HTMLNamedColors
import com.cibo.evilplot.numeric.Bounds
import com.cibo.evilplot.plot.renderers.BarRenderer

#### Import some distributions to generate some sample data

In [ ]:
import org.apache.commons.math3.distribution.GammaDistribution
import org.apache.commons.math3.distribution.NormalDistribution

#### Import data sketching and spline interpolation

In [ ]:
import org.isarnproject.sketches.TDigest
import com.manyangled.snowball.analysis.interpolation.MonotonicSplineInterpolator

#### declare some sampling distributions

In [ ]:
val gamma = new GammaDistribution(1.0, 1.0)
val normal = new NormalDistribution()

#### Synthesize some data and sketch it

In [ ]:
val rawdata = Array.fill(10000) { normal.sample()}
val sketch = TDigest.sketch(rawdata)

#### Collect some (x,y) points from the sketched CDF
These are used as input to the spline fitting routines

In [ ]:
val ydata = (0.0 until 1.0 by 0.01).toArray :+ 1.0
val xdata = ydata.map { y => sketch.cdfInverse(y) }
val (xmin, xmax) = (sketch.cdfInverse(0), sketch.cdfInverse(1))

### Set up an interpolator and fit the data from the CDF sketch
Note that the PDF spline is just the gradient of the CDF spline

In [ ]:
val interpolator = new MonotonicSplineInterpolator()
interpolator.addEqualityConstraint(xmin, 0.0 + 1e-9)
interpolator.addEqualityConstraint(xmax, 1.0 - 1e-9)
interpolator.setBounds(xmin, xmax)
interpolator.setM(20)

val cdfspline = interpolator.interpolate(xdata, ydata)
val pdfspline = cdfspline.polynomialSplineDerivative()

### These wrap the raw splines and define the domain to (-inf, +inf)

In [ ]:
val cdffit = (x: Double) => x match {
    case x if (x <= xmin) => 0.0
    case x if (x >= xmax) => 1.0
    case x => cdfspline.value(x)
}
val pdffit = (x: Double) => x match {
    case x if (x <= xmin) => 0.0
    case x if (x >= xmax) => 0.0
    case x => pdfspline.value(x)
}

### Declare some custom binning functions for EvilPlot
These should probably become a PR

In [ ]:
object customBinners { 
    def cumulative(values: Seq[Double], xbounds: Bounds, binCount: Int): Seq[Point] = 
        doBinning(values, xbounds, binCount, false, true)
    def cumulativeNormalized(values: Seq[Double], xbounds: Bounds, binCount: Int): Seq[Point] = 
        doBinning(values, xbounds, binCount, true, true)
    def density(values: Seq[Double], xbounds: Bounds, binCount: Int): Seq[Point] = {
        val binWidth = xbounds.range / binCount
        doBinning(values, xbounds, binCount, true, false).map { case Point(x,y) => Point(x, y/binWidth) }
    }

    private def doBinning(values: Seq[Double], xbounds: Bounds, binCount: Int,
                              normalize: Boolean, cumulative: Boolean): Seq[Point] = {
        val binWidth = xbounds.range / binCount
        val grouped = values.groupBy { value => 
            math.min(((value - xbounds.min) / binWidth).toInt, binCount - 1)
        }
        val pts = (0 until binCount).flatMap { i =>
            grouped.get(i).map { vs =>
                val y = (if (normalize) vs.size.toDouble / values.size else vs.size)
                val x = i * binWidth + xbounds.min
                Point(x, y)
            }
        }
        if (!cumulative) pts else {
            pts.scanLeft(Point(0,0)) { case (Point(_, t), Point(x,y)) => Point(x, y + t) }.drop(1)
        }
    }
}

### Compare a cumulative histogram, the "raw" sketch CDF and a cubic spline
* Green bars are cumulative histogram of data
* Red line is the CDF taken directly from the t-digest sketch
* Blue line is the spline that was fit to the raw CDF data

In [ ]:
val histplot = Histogram(
    rawdata,
    barRenderer = Some(BarRenderer.default(Some(HTMLNamedColors.green.copy(opacity = 0.25)))),
    binningFunction = customBinners.cumulativeNormalized)
val cdfplot = FunctionPlot.series((x:Double) => sketch.cdf(x), "cdf(x)", HTMLNamedColors.red, xbounds = Some(Bounds(xmin, xmax)))
val splineplot = FunctionPlot.series((x:Double) => cdffit(x), "spline(x)", HTMLNamedColors.dodgerBlue, xbounds = Some(Bounds(xmin, xmax)))
val plt = Overlay(histplot, cdfplot, splineplot).overlayLegend(x=0.6).render()
displayPlot(plt)

### Compare a "density" histogram, the "raw" sketch PDF and a cubic spline of PDF
* Green bars are raw histogram of data
* Red line is the PDF estimated directly from the t-digest sketch
* Blue line is the gradient of the CDF spline above

In [ ]:
val histplot = Histogram(
    rawdata,
    barRenderer = Some(BarRenderer.default(Some(HTMLNamedColors.green.copy(opacity = 0.25)))),
    binningFunction = customBinners.density)
val cdfplot = FunctionPlot.series((x:Double) => (sketch.cdf(x+0.01) - sketch.cdf(x))/0.01, "cdf-gradient-estimate", HTMLNamedColors.red, xbounds = Some(Bounds(xmin, xmax)))
val splineplot = FunctionPlot.series((x:Double) => pdffit(x), "spline-cdf-gradient", HTMLNamedColors.dodgerBlue, xbounds = Some(Bounds(xmin, xmax)))
val plt = Overlay(histplot, cdfplot, splineplot).overlayLegend(x=0.8).render()
displayPlot(plt)